Necesito crear un sistema de recomedación que introduzaca un id y retorne 5 videojuegos recomendados

In [60]:
# Cargar el conjunto de datos
df1 = pd.read_csv('../Data/games.csv')
df = df1[['genres', 'app_name','tags','id']].copy()
df = df.dropna(subset=['id'])
df['id'] = df['id'].astype(int)

# Preprocesar los datos
df = df.dropna()

# Eliminar los corchetes de la columna 'genres'
df['genres'] = df['genres'].apply(lambda x: ', '.join(x)).str.replace('[','').str.replace(']','')

# Crea una instancia del vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Aplica TF-IDF a las características
X = vectorizer.fit_transform(df[['genres', 'app_name', 'tags']].apply(lambda x: ' '.join(x), axis=1))

# Calcula la similitud del coseno entre los vectores resultantes
cosine_sim = cosine_similarity(X)

def recommend_games(target_game_id, similarity_matrix, game_data, num_recommendations=5):
    # Obtener el índice del juego que coincide con el ID
    idx = game_data[game_data['id'] == target_game_id].index[0]

    # Obtener las puntuaciones de similitud del juego con todos los demás juegos
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # Ordenar los juegos según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las recomendaciones
    sim_scores = sim_scores[1:num_recommendations+1]
    game_indices = [i[0] for i in sim_scores]
    return game_data['app_name'].iloc[game_indices].tolist()

    # Crear un DataFrame vacío para almacenar las recomendaciones
    recommendations_df = pd.DataFrame(columns=['id', 'recomendaciones'])

    # Iterar sobre todos los IDs de los juegos y obtener las recomendaciones
    for game_id in df['id']:
        recommendations = recommend_games(game_id, cosine_sim, df)
        recommendations_df = pd.concat([recommendations_df, pd.DataFrame({'id': game_id, 'recomendaciones': [recommendations]})])

    

In [61]:
recommendations_df

,id,recomendaciones
0,761140,"[Summoner, Trapped Summoner, I'm Lost, Dungeon..."
0,643980,"[Children of Zodiarcs, Infinity Wars: Animated..."
0,670290,"[3D Pool, Pure Pool, Pool Nation FX Lite, Virt..."
0,767400,"[D.W.A.R.F.S., Project: R.E.B.O.O.T 2, Project..."
0,772540,"[SNOW - All Access Legend Pass, SNOW - All Acc..."
...,...,...
0,299540,"[Land It!, Slash It, Land it Rocket, Slash It ..."
0,305200,"[Colony, Colony, Mars Colony:Challenger, Aven ..."
0,307420,"[LOGistICAL, Africa Hunting, LOGistICAL: Japan..."
0,304140,"[Roads of Rome 3, Roads of Rome 2, Roads of Ro..."


In [59]:
recommendations_df.to_csv('recomendacion.csv', index=False)

In [62]:
df_recomendacion =pd.read_csv('recomendacion.csv')

In [ ]:
def get_recommendations(game_id):
    # Cargar el archivo CSV con las recomendaciones
    recommendations_df = pd.read_csv('recomendaciones.csv')

    # Obtener las recomendaciones para el ID especificado
    recommendations = recommendations_df[recommendations_df['id'] == game_id]['recomendaciones'].iloc[0]

    return recommendations